In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import plotly.express as px
import seaborn as sns

# Import Cleaned data

In [ ]:
df = pd.read_csv('../Cleaned Data Set/cleaned_data_set_NO_anomalies.csv', index_col='timestamp')

In [ ]:
df.head()

# Basic Data Description

### Group data by Site and Primary Use

In [ ]:
sitbuildloc = df.columns.to_series().str.extract(r'(?P<site>[^_]+)_(?P<usage>[^_]+)_(?P<location>.+)')
sitbuildloc.index = df.columns

sitbuildloc

In [ ]:
total_kwh = df.sum(axis=0)
sitbuildloc['total_kwh'] = total_kwh

In [ ]:
site_usage_counts = sitbuildloc.groupby(['site', 'usage']).size()
site_counts = sitbuildloc.groupby(['site']).size()
usage_counts = sitbuildloc.groupby(['usage']).size()

In [ ]:
# Melt for long-form
df_EDA = df.copy()
df_EDA['timestamp'] = df_EDA.index
df_melted = df_EDA.melt(id_vars='timestamp', var_name='column', value_name='value')
df_melted = df_melted.join(sitbuildloc, on='column')

usage_mean_df = (
    df_melted.groupby(['timestamp', 'usage'])['value']
    .mean()
    .reset_index()
)

site_mean_df = (
    df_melted.groupby(['timestamp', 'site'])['value']
    .mean()
    .reset_index()
)

usage_total_df = (
    df_melted.groupby(['timestamp', 'usage'])['value']
    .sum()
    .reset_index()
)

site_total_df = (
    df_melted.groupby(['timestamp', 'site'])['value']
    .sum()
    .reset_index()
)

site_usage_mean_df = (
    df_melted.groupby(['timestamp', 'site', 'usage'])['value']
    .mean()
    .reset_index()
)

# Description of Data

## By Site

In [ ]:
pivot_site_mean_df = usage_mean_df.pivot(index='timestamp', columns='usage', values='value')
pivot_site_mean_df.describe()

In [ ]:
pivot_site_total_df = usage_total_df.pivot(index='timestamp', columns='usage', values='value')
pivot_site_total_df.describe()

## By Primary Use

In [ ]:
pivot_usage_mean_df = usage_mean_df.pivot(index='timestamp', columns='usage', values='value')
pivot_usage_mean_df.describe()

In [ ]:
pivot_usage_total_df = usage_total_df.pivot(index='timestamp', columns='usage', values='value')
pivot_usage_total_df.describe()

## Investigate data by Site

In [ ]:
site_counts_df = site_counts.reset_index()
site_counts_df.columns = ['Site', 'Number of Buildings']
fig = px.bar(site_counts_df, x='Site', y = 'Number of Buildings', title = 'Number of Building per Site')
fig.show()

fig.write_image(f"../EDA/eda_images/num_per_sites.png")

### Mean Energy Use per Building by Site

In [ ]:
building_mean_kwh = sitbuildloc.groupby('site')['total_kwh'].mean()
building_mean_kwh_df = building_mean_kwh.reset_index()
building_mean_kwh_df.columns = ['Site', 'Mean Energy Use']
fig = px.bar(building_mean_kwh_df, x='Site', y = 'Mean Energy Use', title = 'Mean Energy Use per Building by Site')
# fig.show()

fig.write_image(f"../EDA/eda_images/mean_energy_per_sites.png")

### Total Energy Use by Site

In [ ]:
building_total_kwh = sitbuildloc.groupby('site')['total_kwh'].sum()
building_total_kwh_df = building_total_kwh.reset_index()
building_total_kwh_df.columns = ['Site', 'Total Energy Use']
fig = px.bar(building_total_kwh_df, x='Site', y = 'Total Energy Use', title = 'Total Energy by Site')
# fig.show()
fig.write_image(f"../EDA/eda_images/total_energy_per_sites.png")

## Investigate data by Primary Use

In [ ]:
usage_counts_df = usage_counts.reset_index()
usage_counts_df.columns = ['Usage', 'Number of Buildings']
fig = px.bar(usage_counts_df, x='Usage', y = 'Number of Buildings', title = 'Number of Building per Primary Use')
fig.show()
fig.write_image(f"../EDA/eda_images/num_per_primary_use.png")

### Mean Energy Use per Building by Primary Use

In [ ]:
building_mean_kwh = sitbuildloc.groupby('usage')['total_kwh'].mean().sort_values(ascending=False)
building_mean_kwh_df = building_mean_kwh.reset_index()
building_mean_kwh_df.columns = ['Primary Use', 'Mean Energy Use']
fig = px.bar(building_mean_kwh_df, x='Primary Use', y = 'Mean Energy Use', title = 'Mean Energy Use per Building by Primary Use')
# fig.show()
fig.write_image(f"../EDA/eda_images/mean_energy_per_primary_use.png")

### Total Energy Use by Primary Use

In [ ]:
building_total_kwh = sitbuildloc.groupby('usage')['total_kwh'].sum()
building_total_kwh_df = building_total_kwh.reset_index()
building_total_kwh_df.columns = ['Primary Use', 'Total Energy Use']
fig = px.bar(building_total_kwh_df, x='Primary Use', y = 'Total Energy Use', title = 'Total Energy by Primary Use')
# fig.show()
fig.write_image(f"../EDA/eda_images/total_energy_per_primary_use.png")

## Boxplot for checking Outliers

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 6))  # 1 row, 2 columns

sns.boxplot(data=sitbuildloc, x='site', y='total_kwh', ax=axes[0])
axes[0].set_title('Boxplot of Building Energy Use categorized Site')
axes[0].set_xlabel('Site')
axes[0].set_ylabel('Energy Use')

sns.boxplot(data=sitbuildloc, x='usage', y='total_kwh', ax=axes[1])
axes[1].set_title('Boxplot by Energy Use categorized Primary Use')
axes[1].set_xlabel('Primary Use')
axes[1].tick_params(axis='x', rotation=45)
axes[1].set_ylabel('Energy Use')
plt.savefig(f"../EDA/eda_images/boxplot.png")
plt.show()


In [ ]:
buildings_to_check = sitbuildloc[(((sitbuildloc['usage']=='public') & (sitbuildloc['total_kwh'] > 9000000)) |
                                    ((sitbuildloc['usage']=='education') & (sitbuildloc['total_kwh'] > 25000000)) |
                                    ((sitbuildloc['usage']=='office') & (sitbuildloc['total_kwh'] > 20000000)) |
                                    ((sitbuildloc['usage']=='assembly') & (sitbuildloc['total_kwh'] > 10000000)))]

In [ ]:
buildings_to_check

### Check the outliers in boxplot

In [ ]:
def plot_special_meter(building):
    usage = re.search(r'.?_(.+)_.?', building).group(1)
    sub_usage_df = usage_mean_df[usage_mean_df['usage']==usage].reset_index(drop=True)
    sub_usage_df['usage'] = 'mean of ' + sub_usage_df['usage']
    
    site = re.search(r'(.+)_.+_.+', building).group(1)
    sub_site_df = site_mean_df[site_mean_df['site']==site].reset_index(drop=True)
    sub_site_df['usage'] = 'mean of ' + sub_site_df['site']
    
    sub_site_usage_df = site_usage_mean_df[((site_usage_mean_df['site']==site) & (site_usage_mean_df['usage']==usage))].reset_index(drop=True)
    sub_site_usage_df['usage'] = 'mean of ' + sub_site_usage_df['site'] + " " + sub_site_usage_df['usage']
    
    meter_time_series = df[building]
    meter_df = meter_time_series.reset_index()
    meter_df.set_index('timestamp', inplace=True)
    meter_df.columns = ['value']
    meter_df['value'] = meter_df['value'].rolling(window=24, min_periods=1).mean()
    meter_df.reset_index(inplace=True)
    meter_df['usage'] = building

    meter_df = pd.concat([meter_df,sub_usage_df, sub_site_df, sub_site_usage_df], axis=0)

    fig = px.line(
        meter_df,
        x='timestamp',
        y='value',
        color='usage',
        title=f'Mean Energy Use Over Time {building} (24-Hour Rolling Avg)'
    )
    fig.write_image(f"../EDA/eda_images/check_specific_meters/building_{building}.png")

In [ ]:
for i, building in enumerate(buildings_to_check.index):
    plot_special_meter(building)

In [ ]:
building_mean_kwh = sitbuildloc.groupby('site')['total_kwh'].mean().sort_values(ascending=False)
building_mean_kwh_df = building_mean_kwh.reset_index()
building_mean_kwh_df.columns = ['Site', 'Mean Energy Use']
fig = px.bar(building_mean_kwh_df, x='Site', y = 'Mean Energy Use', title = 'Mean Energy Use per Building by Site')
fig.show()

# Trend

In [ ]:
def plot_trend(df, attr, agg_val, data_type=None):
    smoothed_frames = []
    
    
    bldg_list =  [data_type] if data_type else df[attr].unique()
    data_type  = data_type if data_type else 'all' 

    for bldg_type in bldg_list:
        sub_df = df[df[attr] == bldg_type].copy()
        sub_df = sub_df.set_index('timestamp').sort_index()

        sub_df['value'] = sub_df['value'].rolling(window=24, min_periods=1).mean()
        sub_df = sub_df.reset_index()
        sub_df[attr] = bldg_type

        smoothed_frames.append(sub_df)
    mean_by_time_building_smoothed = pd.concat(smoothed_frames, ignore_index=True)

    if attr == 'site':
        title_attr = 'Site'
    else:
        title_attr = 'Primary Use'

    fig = px.line(
        mean_by_time_building_smoothed,
        x='timestamp',
        y='value',
        color=attr,
        title=f'{agg_val} Energy Use Over Time by {title_attr} (24-Hour Rolling Avg)'
    )
    fig.update_layout(legend_title_text=attr)
    
    
    fig.write_image(f"../EDA/eda_images/trend/{agg_val}_Energy_{attr}_{data_type}.png")
    # if data_type=='all':
        # fig.show()

## Mean Energy Use Over Time by Primary Use (24-Hour Rolling Avg)

In [ ]:
plot_trend(usage_mean_df, 'usage', 'Mean')

In [ ]:
smoothed_frames = []

for bldg_type in usage_counts[usage_counts<8].index.to_list():
    sub_df = usage_mean_df[usage_mean_df['usage'] == bldg_type].copy()
    sub_df = sub_df.set_index('timestamp').sort_index()

    sub_df['value'] = sub_df['value'].rolling(window=24, min_periods=1).mean()
    sub_df = sub_df.reset_index()
    sub_df['usage'] = bldg_type

    smoothed_frames.append(sub_df)
mean_by_time_building_smoothed = pd.concat(smoothed_frames, ignore_index=True)

fig = px.line(
    mean_by_time_building_smoothed,
    x='timestamp',
    y='value',
    color='usage',
    title='Mean Energy Use Over Time by Primary Use (24-Hour Rolling Avg)'
)
fig.update_layout(legend_title_text='Building Type')
fig.show()

fig.write_image(f"../EDA/eda_images/trend/mean_Energy_usage_low_count.png")


In [ ]:
smoothed_frames = []

for bldg_type in ['food', 'other', 'religion', 'retail', 'science', 'services', 'warehouse']:
    sub_df = usage_mean_df[usage_mean_df['usage'] == bldg_type].copy()
    sub_df = sub_df.set_index('timestamp').sort_index()

    sub_df['value'] = sub_df['value'].rolling(window=24, min_periods=1).mean()
    sub_df = sub_df.reset_index()
    sub_df['usage'] = bldg_type

    smoothed_frames.append(sub_df)
mean_by_time_building_smoothed = pd.concat(smoothed_frames, ignore_index=True)

fig = px.line(
    mean_by_time_building_smoothed,
    x='timestamp',
    y='value',
    color='usage',
    title='Mean Energy Use Over Time by Primary Use (24-Hour Rolling Avg)'
)
fig.update_layout(legend_title_text='Building Type')
fig.show()

fig.write_image(f"../EDA/eda_images/trend/mean_Energy_usage_other.png")

# Update Primary Use Categories

## Filter Primary Use = 'religion' and 
## Combine Primary Use ['food', 'other', 'retail', 'science', 'services', 'warehouse'] as Primary Use = 'other'

In [ ]:
filtered_df = df.drop(columns=[col for col in df.columns if '_religion_' in col])

def replace_words(col):
    for word in ['food', 'retail', 'science', 'services', 'warehouse']:
        col = col.replace(word, 'other')
    return col

filtered_df.columns = [replace_words(col) for col in filtered_df.columns]

In [ ]:
filtered_sitbuildloc = filtered_df.columns.to_series().str.extract(r'(?P<site>[^_]+)_(?P<usage>[^_]+)_(?P<location>.+)')
filtered_sitbuildloc.index = filtered_df.columns

# Group data by Site and Primary Use

In [ ]:
total_kwh = filtered_df.sum(axis=0)
filtered_sitbuildloc['total_kwh'] = total_kwh

In [ ]:
site_usage_counts = filtered_sitbuildloc.groupby(['site', 'usage']).size()
site_counts = filtered_sitbuildloc.groupby(['site']).size()
usage_counts = filtered_sitbuildloc.groupby(['usage']).size()

In [ ]:
usage_counts

In [ ]:
# Melt for long-form
df_EDA = filtered_df.copy()
df_EDA['timestamp'] = df_EDA.index
df_melted = df_EDA.melt(id_vars='timestamp', var_name='column', value_name='value')
df_melted = df_melted.join(sitbuildloc, on='column')

usage_mean_df = (
    df_melted.groupby(['timestamp', 'usage'])['value']
    .mean()
    .reset_index()
)

site_mean_df = (
    df_melted.groupby(['timestamp', 'site'])['value']
    .mean()
    .reset_index()
)

usage_total_df = (
    df_melted.groupby(['timestamp', 'usage'])['value']
    .sum()
    .reset_index()
)

site_total_df = (
    df_melted.groupby(['timestamp', 'site'])['value']
    .sum()
    .reset_index()
)

site_usage_mean_df = (
    df_melted.groupby(['timestamp', 'site', 'usage'])['value']
    .mean()
    .reset_index()
)

# Reinvestigate Data by Site

In [ ]:
site_counts_df = site_counts.reset_index()
site_counts_df.columns = ['Site', 'Number of Buildings']
fig = px.bar(site_counts_df, x='Site', y = 'Number of Buildings', title = 'Number of Building per Site')
fig.show()

fig.write_image(f"../EDA/eda_images/num_per_sites_new.png")

In [ ]:
building_total_kwh = filtered_sitbuildloc.groupby('site')['total_kwh'].sum()
building_total_kwh_df = building_total_kwh.reset_index()
building_total_kwh_df.columns = ['Site', 'Total Energy Use']
fig = px.bar(building_total_kwh_df, x='Site', y = 'Total Energy Use', title = 'Total Energy by Site')
fig.show()
fig.write_image(f"../EDA/eda_images/total_energy_per_sites_new.png")

# Reinvestigate Data by Primary Use

In [ ]:
usage_counts_df = usage_counts.reset_index()
usage_counts_df.columns = ['Usage', 'Number of Buildings']
fig = px.bar(usage_counts_df, x='Usage', y = 'Number of Buildings', title = 'Number of Building per Primary Use')
fig.show()
fig.write_image(f"../EDA/eda_images/num_per_primary_use_new.png")

In [ ]:
building_total_kwh = filtered_sitbuildloc.groupby('usage')['total_kwh'].sum()
building_total_kwh_df = building_total_kwh.reset_index()
building_total_kwh_df.columns = ['Primary Use', 'Total Energy Use']
fig = px.bar(building_total_kwh_df, x='Primary Use', y = 'Total Energy Use', title = 'Total Energy by Primary Use')
# fig.show()
fig.write_image(f"../EDA/eda_images/total_energy_per_primary_use_new.png")

# Trend

## Trend by Primary Use

In [ ]:
plot_trend(usage_mean_df, 'usage', 'Mean')

In [ ]:
plot_trend(usage_total_df, 'usage', 'Total')

### Trend by each Primary Type

In [ ]:
for usage in usage_total_df['usage'].unique():
    plot_trend(usage_total_df, 'usage', 'Total', usage)

In [ ]:
for usage in usage_total_df['usage'].unique():
    plot_trend(usage_mean_df, 'usage', 'Mean', usage)

## Trend by Site

In [ ]:
plot_trend(site_mean_df, 'site', 'Mean')

In [ ]:
plot_trend(site_total_df, 'site', 'Total')

In [ ]:
for site in site_mean_df['site'].unique():
    plot_trend(site_mean_df, 'site', 'Mean', site)

In [ ]:
for site in site_total_df['site'].unique():
    plot_trend(site_total_df, 'site', 'Total', site)